# DESaster Vers -  0.1

In [21]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, config, io, movement, search, rebuild


##### instantiate a simpy environment

In [22]:
simulation = simpy.Environment()

In [23]:
def find_rebuild_money(simulation, human_capital, financial_capital, entity):
    try:
        
        patience_length = 29.0
        start_money_search = simulation.now
        patience_end = simulation.now + patience_length
        
        if entity.insurance > 0.0:
        
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_insurance = simulation.process(request.insurance_claim(simulation, human_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_insurance
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                
                try_insurance.interrupt(simulation.now - start_money_search)


                return
        
        if entity.money_to_rebuild < entity.residence.damage_value:
            
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_fema = simulation.process(request.fema_assistance(simulation, human_capital, 
                                                                  financial_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_fema
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                try_fema.interrupt(simulation.now - start_money_search)

                return
            
        if entity.money_to_rebuild < entity.residence.damage_value:
            
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_loan = simulation.process(request.loan(simulation, human_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_loan
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                try_loan.interrupt(simulation.now - start_money_search)

                return
            
            
        entity.story.append(
                                'It took {0} {1} days to exhaust the search for money. '.format(
                                entity.name, simulation.now - start_money_search))

        return
    
    except:
        
        entity.story.append(
                            '{0} gave up searching for money after {1} days. '.format(
                                entity.name, simulation.now - start_money_search))
        
        return

In [24]:
def master_process(simulation, human_capital, financial_capital, entity):
       
    yield simulation.process(request.inspection(simulation, human_capital, entity))
    
    if entity.money_to_rebuild < entity.residence.damage_value:
        rebuild_money_search = simulation.process(find_rebuild_money(simulation, human_capital, 
                                                                     financial_capital, entity))

    
    if entity.money_to_rebuild >= entity.residence.damage_value:
        yield simulation.process(request.engineering_assessment(simulation, human_capital, entity))
        
        yield simulation.process(request.permit(simulation, human_capital, entity))
        
        yield simulation.process(rebuild.rebuild_house(simulation, human_capital, entity))
        
        return
   

In [25]:
human_cap_data = {
        "inspectors": 1,
        "insurance adjusters": 1,
        "fema processors": 1,
        "permit processors": 1,
        "contractors": 1,
        "loan processors": 1,
        "engineers": 1
                 }

human_capital = capitals.HumanCapital(simulation, human_cap_data) 

In [26]:
financial_cap_data = {"fema aid": 350000000}
# financial_cap_data = {"fema aid": 0}

financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

In [27]:
file_path = "../inputs/household_inputs_test.csv"
households_df = pd.read_csv(file_path)
households_df.head()

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Damage Value
0,Alfred,30000,10000,0,62 That St,Mobile,100,1,1,700,1920,100000,None,20000
1,Bruce,100000,1000000,10000000,720 This Rd,Single Family,100000,6,5,5000,1920,10000000,Slight,50000
2,Selena,10000,100,0,1001 Other Ave,Multi Family,10,0,1,250,1960,10000,Complete,8000
3,Fish,50000,1000,550000,26000 Out There Lane,Single Family,2000,4,2,2000,2010,800000,Moderate,650000


In [28]:
household = {}
for i in households_df.index:
    household[i] = entities.Household(simulation, households_df.iloc[i])

In [29]:
for i in range(len(household)):
    #print (household[i])
    simulation.process(master_process(simulation, human_capital, financial_capital, household[i]))

In [30]:
simulation.run()

AttributeError: 'Household' object has no attribute 'request'

In [ ]:
household[0].story

In [ ]:
household[1].story

In [ ]:
household[2].story

In [ ]:
household[3].story